In [ ]:
#Standard inmorts 
import numpy as np
import pandas as pd
import os
#import feather
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [ ]:
# Bestellplanung
df_a1 = pd.read_feather('../Data3/D3_EBAN_CL.ftr', columns=None, use_threads=True)
# Bestellfreigabe / Bestellung
df_a2 = pd.read_feather('../Data3/D3_EK_CL.ftr', columns=None, use_threads=True)
# Wareneingang
df_a3 = pd.read_feather('../Data3/D3_MB_XS_CL.ftr', columns=None, use_threads=True)
# Rechnungserfassung
df_a4 = pd.read_feather('../Data3/D3_RE_CL.ftr', columns=None, use_threads=True)
# Kreditorische Gutschrift / Rechnungsbegleichung
df_a5 = pd.read_feather('../Data3/D3_BEL_CL.ftr', columns=None, use_threads=True)


In [ ]:

# Falls wir Datum / Timestamp dazulesen müssen
df_cd = pd.read_feather('../Data3/D3_CDHDR-CDPOS_raw.ftr', columns=None, use_threads=True)

In [ ]:
df_a1.shape

In [ ]:
df_a2.shape

In [ ]:
df_a3.shape

In [ ]:
df_a4.shape

In [ ]:
df_a5.shape

In [ ]:
df_cd.shape

In [ ]:
df_a5.dtypes

# 1 .Add Time to A1 and A2 hence theres no time in DB Table 

In [ ]:
df_cd.head(2)

In [ ]:
df_cd.shape

## for EBAN

In [ ]:
df_a1 = df_a1.drop([254, 255, 256, 257]).copy()

In [ ]:
df_a1.shape

In [ ]:
df_cd_eban = df_cd.loc[(df_cd['TABNAME'] == 'EKKO')].copy()

In [ ]:
df_cd_eban.shape

In [ ]:
df_cd_eban.head(2)

In [ ]:
# changes for joining the data
#dtype changes CHPOS
freq = 'm'
df_cd_eban['TMSTMP'] = df_cd_eban['TMSTMP'] - pd.Timedelta(1, unit=freq)

df_cd_eban['MergeKey'] = df_cd_eban['OBJECTID_x'].astype(str)
df_cd_eban['MergeKey'] = df_cd_eban['MergeKey'].astype(float)
df_cd_eban['MergeKey'] = df_cd_eban['MergeKey'].fillna(0)
df_cd_eban['MergeKey'] = df_cd_eban['MergeKey'].astype(int)


#dtype Changes EBAN
df_a1['MergeKey'] = df_a1['EBELN'].astype(str).str[:10]
df_a1['MergeKey'] = df_a1['MergeKey'].astype(float)
df_a1['MergeKey'] = df_a1['MergeKey'].fillna(0)
df_a1['MergeKey'] = df_a1['MergeKey'].astype(int)

In [ ]:
df_a1['EBELN'] = df_a1['EBELN'].astype(str).str[:10].astype(int)
df_a1['EBELP'] = df_a1['EBELP'].astype(int)

In [ ]:
df_cd_eban.shape

In [ ]:
df_cd_eban = df_cd_eban[['MergeKey','TMSTMP']].copy()

In [ ]:
df_cd_eban.head(2)

In [ ]:
print(df_cd_eban.shape)
print(df_a1.shape)

In [ ]:
df_a1_1 = pd.merge(df_a1, df_cd_eban, how='left', on='MergeKey')

In [ ]:
df_a1_1.head(5)

In [ ]:
df_a1_1['EBELN'] = df_a1_1['EBELN'].astype(str).str[:10].astype(int)
df_a1_1['EBELP'] = df_a1_1['EBELP'].astype(int)

In [ ]:
df_a1_1.head(5)

In [ ]:
print(df_a1_1.shape)

## for EKKO

In [ ]:
df_cd_ekko = df_cd.loc[(df_cd['TABNAME'] == 'EKKO')].copy()


In [ ]:
df_cd_ekko.shape

In [ ]:
# changes for joining the data
#dtype changes CHPOS
df_cd_ekko['MergeKey'] = df_cd_ekko['OBJECTID_x'].astype(str)
df_cd_ekko['MergeKey'] = df_cd_ekko['MergeKey'].astype(float)
df_cd_ekko['MergeKey'] = df_cd_ekko['MergeKey'].fillna(0)
df_cd_ekko['MergeKey'] = df_cd_ekko['MergeKey'].astype(int)

#dtype Changes EBAN
df_a2['MergeKey'] = df_a2['EBELN']

In [ ]:
df_cd_ekko = df_cd_ekko[['MergeKey','TMSTMP']].copy()

In [ ]:
print(df_cd_eban.shape)
print(df_a1.shape)

In [ ]:
#df['TMSTMP'] = pd.to_datetime(df['CPUDT'].astype(str) + df['CPUTM'].astype(str) , format='%Y-%m-%d%H:%M:%S')
df_a2_1 = pd.merge(df_a2, df_cd_ekko, how='left', on='MergeKey')

# 2. Create A Event Log

## 2.1  Purchase Requisition

In [ ]:
# CaseID 	Activity 	Timestamp + unsere Cluster als Activity

In [ ]:
df_a1_1['CaseID'] = df_a1_1['MergeKey'].astype(str) + df_a1_1['EBELP'].astype(str)
df_a1_1['Activity'] = 'Create_Purchase_Requisition'
df_a1_1['Activity_kproto'] = 'Create_Purchase_Requisition_' + df_a1_1['kproto_labels'].astype(str)
df_a1_1['Activity_dbscan'] = 'Create_Purchase_Requisition_' + df_a1_1['dbscan_gower'].astype(str)
df_a1_1['Activity_Aglo_Sgl'] = 'Create_Purchase_Requisition_' + df_a1_1['AgloSngl_gower'].astype(str)
df_a1_1['Activity_Aglo_Avg'] = 'Create_Purchase_Requisition_' + df_a1_1['AgloAvg_gower'].astype(str)
df_a1_1['Activity_Aglo_Cmp'] = 'Create_Purchase_Requisition_' + df_a1_1['AgloComp_gower'].astype(str)


In [ ]:
df1 = df_a1_1[['CaseID', 'TMSTMP',  'Activity', 'Activity_kproto', 'Activity_dbscan', 'Activity_Aglo_Sgl', 'Activity_Aglo_Avg', 'Activity_Aglo_Cmp']].copy()
df1.shape

In [ ]:
df2.Activity_Aglo_Cmp.nunique()

In [ ]:
df1["Activity_BOB"] = df1["Activity_Aglo_Cmp"]

In [ ]:
df1.head(3)

## 2.2 Create Purchase Orders

In [ ]:
df_a2_1['CaseID'] = df_a2_1['EBELN'].astype(str) + df_a2_1['EBELP'].astype(str)
df_a2_1['Activity'] = 'Create_Purchase_Order'
df_a2_1['Activity_kproto'] = 'Create_Purchase_Order_' + df_a2_1['kproto_labels'].astype(str)
df_a2_1['Activity_dbscan'] = 'Create_Purchase_Order_' + df_a2_1['dbscan_gower'].astype(str)
df_a2_1['Activity_Aglo_Sgl'] = 'Create_Purchase_Order_' + df_a2_1['AgloSngl_gower'].astype(str)
df_a2_1['Activity_Aglo_Avg'] = 'Create_Purchase_Order_' + df_a2_1['AgloAvg_gower'].astype(str)
df_a2_1['Activity_Aglo_Cmp'] = 'Create_Purchase_Order_' + df_a2_1['AgloComp_gower'].astype(str)

In [ ]:
df_a2_1.head()

In [ ]:
df2 = df_a2_1[['CaseID', 'TMSTMP',  'Activity', 'Activity_kproto', 'Activity_dbscan', 'Activity_Aglo_Sgl', 'Activity_Aglo_Avg', 'Activity_Aglo_Cmp']].copy()
df2.shape

In [ ]:
df2.Activity_dbscan.nunique()

In [ ]:
df2["Activity_BOB"] = df2["Activity_dbscan"]

In [ ]:
df2.head(2)

## 2.3 GR

In [ ]:
df_a3['MergeKey'] = df_a3['EBELN'].astype(str).str[:10]
df_a3['MergeKey'] = df_a3['MergeKey'].astype(float)
df_a3['MergeKey'] = df_a3['MergeKey'].fillna(0)
df_a3['MergeKey'] = df_a3['MergeKey'].astype(int)


In [ ]:
df_a3['EBELP2'] = df_a3['EBELP'].astype(int)

In [ ]:
df_a3['CaseID'] = df_a3['MergeKey'].astype(str) + df_a3['EBELP2'].astype(str)
df_a3['Activity'] = 'Goods_Receipt'
df_a3['Activity_kproto'] = 'Goods_Receipt_' + df_a3['kproto_labels'].astype(str)
df_a3['Activity_dbscan'] = 'Goods_Receipt_' + df_a3['dbscan_gower'].astype(str)
df_a3['Activity_Aglo_Sgl'] = 'Goods_Receipt_' + df_a3['AgloSngl_gower'].astype(str)
df_a3['Activity_Aglo_Avg'] = 'Goods_Receipt_' + df_a3['AgloAvg_gower'].astype(str)
df_a3['Activity_Aglo_Cmp'] = 'Goods_Receipt_' + df_a3['AgloComp_gower'].astype(str)

In [ ]:
df_a3['EBELN'] = df_a3['EBELN'].astype(str).str[:10].astype(int)
df_a3['EBELP'] = df_a3['EBELP'].astype(int)

In [ ]:
df3 = df_a3[['CaseID', 'TMSTMP',  'Activity', 'Activity_kproto', 'Activity_dbscan', 'Activity_Aglo_Sgl', 'Activity_Aglo_Avg', 'Activity_Aglo_Cmp']].copy()
df3.shape

In [ ]:
df_a3.head(2)

In [ ]:
df3["Activity_BOB"] = df3["Activity_Aglo_Cmp"]

In [ ]:
df3.head(2)

# 2.4 Record Invoice

In [ ]:
df_a4.shape

In [ ]:
df_a4.head(2)

In [ ]:
df_a4['EBELN'] = df_a4['EBELN'].astype(int)
df_a4['MergeKey'] = df_a4['EBELN']

In [ ]:
df_a4['EBELP'] = df_a4['EBELP'].astype(int)

In [ ]:
df_a4['CaseID'] = df_a4['MergeKey'].astype(str) + df_a4['EBELP'].astype(str)
df_a4['Activity'] = 'Record_Invocie'
df_a4['Activity_kproto'] = 'Record_Invocie_' + df_a4['kproto_labels'].astype(str)
df_a4['Activity_dbscan'] = 'Record_Invocie_' + df_a4['dbscan_gower'].astype(str)
df_a4['Activity_Aglo_Sgl'] = 'Record_Invocie_' + df_a4['AgloSngl_gower'].astype(str)
df_a4['Activity_Aglo_Avg'] = 'Record_Invocie_' + df_a4['AgloAvg_gower'].astype(str)
df_a4['Activity_Aglo_Cmp'] = 'Record_Invocie_' + df_a4['AgloComp_gower'].astype(str)

In [ ]:
df4 = df_a4[['CaseID', 'TMSTMP',  'Activity', 'Activity_kproto', 'Activity_dbscan', 'Activity_Aglo_Sgl', 'Activity_Aglo_Avg', 'Activity_Aglo_Cmp']].copy()
df4.shape

In [ ]:
df4["Activity_BOB"] = df4["Activity_Aglo_Cmp"]

In [ ]:
df4.head(2)

## 2.5 Clear Invoice

In [ ]:
df_a5.shape

In [ ]:
df_a5.head(10)

In [ ]:
#df_a5['CaseID'] = df_a5['MergeKey'].astype(str) + df_a5['EBELP'].astype(str)
df_a5['Activity'] = 'Clear_Invocie'
df_a5['Activity_kproto'] = 'Clear_Invocie_' + df_a5['kproto_labels'].astype(str)
df_a5['Activity_dbscan'] = 'Clear_Invocie_' + df_a5['dbscan_gower'].astype(str)
df_a5['Activity_Aglo_Sgl'] = 'Clear_Invocie_' + df_a5['AgloSngl_gower'].astype(str)
df_a5['Activity_Aglo_Avg'] = 'Clear_Invocie_' + df_a5['AgloAvg_gower'].astype(str)
df_a5['Activity_Aglo_Cmp'] = 'Clear_Invocie_' + df_a5['AgloComp_gower'].astype(str)

In [ ]:
df_a5['XBLNR'] = df_a5['XBLNR'].astype(str)
df_a5['XBLNR'] = df_a5['XBLNR'].astype(int)

In [ ]:
df_a5.tail(2)

In [ ]:
df_ek = df_a2_1[['EBELN', 'EBELP']].copy()

In [ ]:
df_ek.shape

In [ ]:
df_ek['CaseID_y'] = df_ek['EBELN'].astype(str) + df_ek['EBELP'].astype(str)

In [ ]:
df_ek['XBLNR'] = df_ek['EBELN']
#df_ek['CaseID'] = df_ek['EBELN']

In [ ]:
df_ek.tail(3)

In [ ]:
df_ek.shape

In [ ]:
df_a5.shape

In [ ]:
df5_1 = []

In [ ]:
df5_1 = df_a5.merge(df_ek, how='right', on='XBLNR') #, suffixes=("_x", None)

In [ ]:
df5_1.shape

In [ ]:
df5_1 = df5_1[df5_1['BELNR'].notna()]

In [ ]:
df5_1.sample(5)

In [ ]:
df5_1['CaseID'] = df5_1['CaseID_y']

In [ ]:
df5_1 = df5_1.drop(['CaseID_y'], axis=1).copy() #'EBELN_y', 'EBELP_y', 

In [ ]:
df5_1.rename(columns = {'EBELN_x':'EBELN', 'EBELP_x':'EBELP'}, inplace = True)

In [ ]:
#finished full DF
df5_1.head(2)

In [ ]:
df5 = df5_1[['CaseID', 'TMSTMP',  'Activity', 'Activity_kproto', 'Activity_dbscan', 'Activity_Aglo_Sgl', 'Activity_Aglo_Avg', 'Activity_Aglo_Cmp']].copy()

In [ ]:
df5["Activity_BOB"] = df5["Activity_Aglo_Cmp"]

In [ ]:
df5.shape

In [ ]:
df5.head(10)

# 3. Merge Files to a single Event Log

In [ ]:
df_el = pd.concat([df1, df2, df3, df4, df5]).copy()

In [ ]:
df_el.head(2)

In [ ]:
df_el.dtypes

In [ ]:
df_el['CaseID'] = df_el['CaseID'].astype(str)
df_el['CaseID'] = df_el['CaseID'].astype(int)

In [ ]:
df_el = df_el.reset_index(drop=True).copy()

In [ ]:
df_el.head(1300)

In [ ]:
df_el.to_csv('../Data3/D3_Event_log.csv')

In [ ]:
import os
import pandas as pd
import pm4py

In [ ]:
dataframe = pm4py.format_dataframe(df_el, case_id='CaseID', activity_key='Activity', timestamp_key='TMSTMP')
event_log = pm4py.convert_to_event_log(dataframe)

In [ ]:
#Event Log 2 with Gower and DBSCAN
dataframe2 = pm4py.format_dataframe(df_el, case_id='CaseID', activity_key='Activity_dbscan', timestamp_key='TMSTMP')
event_log2 = pm4py.convert_to_event_log(dataframe2)

In [ ]:
dataframe.to_csv('ev_ohnecluster.csv')

In [ ]:
#Event Log 3 with Gower and kproto
dataframe3 = pm4py.format_dataframe(df_el, case_id='CaseID', activity_key='Activity_kproto', timestamp_key='TMSTMP')
event_log3 = pm4py.convert_to_event_log(dataframe3)

In [ ]:
#Event Log 4 with Gower and Activity_Aglo_Cmp
dataframe4 = pm4py.format_dataframe(df_el, case_id='CaseID', activity_key='Activity_Aglo_Cmp', timestamp_key='TMSTMP')
event_log4 = pm4py.convert_to_event_log(dataframe4)

In [ ]:
#Event Log 4 with Gower and Activity_Aglo_Cmp
dataframe5 = pm4py.format_dataframe(df_el, case_id='CaseID', activity_key='Activity_BOB', timestamp_key='TMSTMP')
event_log5 = pm4py.convert_to_event_log(dataframe5)

In [ ]:
#export for descriptive statistics 
dataframe.to_csv('ev_dbscan.csv')
#dataframe3.to_csv('ev_kproto.csv')
#dataframe4.to_csv('ev_aglo_cmp.csv')

In [ ]:
dataframe.shape

In [ ]:
dataframe.head()

# export danieldf_a5

df_el = df_el.sort_values(['CaseID', 'TMSTMP']).copy()

df_sort2 = df_el.reset_index(drop=True).copy()

maxi = max(df_sort2.index) 
df_sort2['Source'] = 'source'
df_sort2['Target'] = 'target'

for ind in df_sort2.index:
        #whats the last rows lastCase
    if ind+1 <= maxi:
        #source = eigen acitivität
        df_sort2['Source'][ind] = df_sort2['Activity'][ind]
        #target = nächste aktivität oder wenn keine existiert ende
        if df_sort2['CaseID'][ind] == df_sort2['CaseID'][ind+1]:
            df_sort2['Target'][ind] =  df_sort2['Activity'][ind+1]
        else:
            df_sort2['Target'][ind] = 'Ende'
        
    else:
        #last row in df
        df_sort2['Source'][ind] = df_sort2['Activity'][ind]
        df_sort2['Target'][ind] = 'Ende'
        

df_sort2.to_feather('source_target.ftr')
df_a1_1.to_feather('source_Create_Purchase_Requisition.ftr')
df_a2_1.to_feather('source_Create_Purchase_Order.ftr') 
df_a3.to_feather('source_Goods_Receipt.ftr')
df_a4.to_feather('source_Record_Invocie.ftr') 
df5_1.to_feather('source_Clear_Invocie.ftr')

df_expd = pd.concat([df_a1_1, df_a2_1, df_a3, df_a4, df5_1]).reset_index(drop=True).copy()
df_sort = df_expd.sort_values(['CaseID', 'TMSTMP']).copy()
df_sort1 = df_sort.reset_index(drop=True).copy()
print(max(df_sort1.index) )

maxi = max(df_sort1.index) 
df_sort1['Source'] = 'source'
df_sort1['Target'] = 'target'

for ind in df_sort1.index:
        #whats the last rows lastCase
    if ind+1 <= maxi:
        #source = eigen acitivität
        df_sort1['Source'][ind] = df_sort1['Activity'][ind]
        #target = nächste aktivität oder wenn keine existiert ende
        if df_sort1['CaseID'][ind] == df_sort1['CaseID'][ind+1]:
            df_sort1['Target'][ind] =  df_sort1['Activity'][ind+1]
        else:
            df_sort1['Target'][ind] = 'Ende'
        
    else:
        #last row in df
        df_sort1['Source'][ind] = df_sort1['Activity'][ind]
        df_sort1['Target'][ind] = 'Ende'
        

cat_cols=['SOURCE', 'TARGET']
value_cols='MENGE'

# transform df into a source-target pair
for i in range(len(cat_cols)-1):
    if i==0:
        sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
        sourceTargetDf.columns = ['source','target','count']
    else:
        tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
        tempDf.columns = ['source','target','count']
        sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
    sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index() #{'count':'sum'}

# add index for source-target pair
sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))

# Process Mining

In [ ]:
dfg, start_activities, end_activities = pm4py.discover_dfg(event_log)
pm4py.view_dfg(dfg, start_activities, end_activities)
pm4py.save_vis_dfg(dfg, start_activities, end_activities, 'PMM_plain.svg')

In [ ]:
#DBSCAN
dfg, start_activities, end_activities = pm4py.discover_dfg(event_log2)
pm4py.view_dfg(dfg, start_activities, end_activities)

In [ ]:
#performance_dfg, start_activities, end_activities = pm4py.discover_performance_dfg(event_log2)
#pm4py.view_performance_dfg(performance_dfg, start_activities, end_activities)

In [ ]:
#K-PRoto
dfg, start_activities, end_activities = pm4py.discover_dfg(event_log3)
pm4py.view_dfg(dfg, start_activities, end_activities)

In [ ]:
#AGCC
dfg, start_activities, end_activities = pm4py.discover_dfg(event_log4)
pm4py.view_dfg(dfg, start_activities, end_activities)

In [ ]:
#Best of Breed BoB
dfg, start_activities, end_activities = pm4py.discover_dfg(event_log5)
pm4py.view_dfg(dfg, start_activities, end_activities)
pm4py.save_vis_dfg(dfg, start_activities, end_activities, 'PMM_BoB.svg')